<a href="https://colab.research.google.com/github/baloglu321/LLM-s/blob/main/ollama_server_cloudflare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyngrok smolagents[litellm] smolagents -U -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.4/148.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 139.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 26.0 MB/s eta 0:00:00


In [ ]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  38545      0 --:--:-- --:--:-- --:--:-- 38495
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:

#!ollama pull qwen2.5-coder:14b-instruct-fp16 #20/3
#!ollama pull deepseek-coder:33b-instruct #0/20 ~6000
#!ollama pull qwen2.5-coder:32b-instruct-q6_K
#!ollama pull qwen2.5vl:7b
!ollama pull gemma3:27b

In [ ]:
# --- Gerekli Kurulumlar ---
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

from flask import Flask, request, Response
import threading
import requests
import time
import subprocess
import socket
import sys

# --- Konfigürasyon ---
FLASK_PORT = 5000
OLLAMA_HOST = "http://127.0.0.1:11434"

# --- Flask App ---
app = Flask(__name__)

@app.route("/api/<path:path>", methods=["GET", "POST"])
def proxy(path):

    upstream_url = f"{OLLAMA_HOST}/api/{path}"

    try:
        if request.method == "POST":
            # KESİN ÇÖZÜM: Ollama'nın beklediği başlıkları zorla ekle
            # LiteLLM'den gelen eksik başlığı tamamlar.

            # Gelen isteğin body'sini ham bayt olarak alıyoruz
            data = request.get_data()

            resp = requests.post(
                upstream_url,
                data=data,  # Ham veriyi gönder
                # Zorunlu başlığı gönder: Ollama'ya Content-Type'ın JSON olduğunu söylüyoruz
                headers={'Content-Type': 'application/json'},
                stream=True,
                timeout=30
            )
        else:
            # GET istekleri için (örneğin /api/tags)
            resp = requests.get(upstream_url, stream=True, timeout=30)

        # Yanıtı istemciye geri gönder
        return Response(
            resp.iter_content(chunk_size=1024),
            status=resp.status_code,
            content_type=resp.headers.get('Content-Type')
        )
    except requests.exceptions.ConnectionError:
        return Response("Ollama servisine (127.0.0.1:11434) bağlanılamadı.", status=503)
    except Exception as e:
        print(f"Proxy Hata: {e}", file=sys.stderr)
        return Response(f"Dahili Proxy Hatası: {e}", status=500)

# --- Flask Threading ve Cloudflared Mantığı (Aynı Kalıyor) ---

def run_cloudflared_tunnel(port):
    print(f"✅ Flask {port} portunda çalışıyor. Cloudflared tünelini başlatılıyor...")
    command = ["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"]
    p = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    public_url = None
    for line in iter(p.stderr.readline, b''):
        l = line.decode('utf-8').strip()
        if "trycloudflare.com" in l:
            url_match = l.split("https://")
            if len(url_match) > 1:
                public_url = ("https://" + url_match[1]).strip().split(' ')[0].strip()
                print(f"🔗 Cloudflare Tunnel URL: {public_url}")
                try:
                    time.sleep(5)
                    r = requests.get(public_url + "/api/tags", timeout=10)
                    print(f"✅ Keep-alive başarılı (Ollama /api/tags): {r.status_code}")
                except Exception as e:
                    print(f"❌ Keep-alive/API Testi Hata: {e}")
                if public_url:
                    break
    p.wait()

def run_flask():
    print(f"🔥 Flask API {FLASK_PORT} portunda başlıyor...")
    app.run(host='127.0.0.1', port=FLASK_PORT, debug=False, use_reloader=False)

# --- Ana Çalışma Akışı ---
flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()
time.sleep(2)
tunnel_thread = threading.Thread(target=run_cloudflared_tunnel, daemon=True, args=(FLASK_PORT,))
tunnel_thread.start()
try:
    while True:
        time.sleep(60)
except KeyboardInterrupt:
    print("\nProgram sonlandırılıyor...")

--2025-11-19 09:43:03--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb [following]
--2025-11-19 09:43:03--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/8a32f7c6-649c-4f0d-806d-e14c19d0786d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-19T10%3A17%3A06Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b89

Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


✅ Flask 5000 portunda çalışıyor. Cloudflared tünelini başlatılıyor...
🔗 Cloudflare Tunnel URL: https://retrieved-offices-myspace-cooperative.trycloudflare.com


INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 09:43:17] "GET /api/tags HTTP/1.1" 200 -


✅ Keep-alive başarılı (Ollama /api/tags): 200


INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 09:43:45] "POST /api/chat HTTP/1.1" 415 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 09:43:49] "POST /api/chat HTTP/1.1" 415 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 09:43:54] "POST /api/chat HTTP/1.1" 415 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 09:43:58] "POST /api/chat HTTP/1.1" 415 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 09:44:07] "POST /api/chat HTTP/1.1" 415 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 09:44:17] "POST /api/chat HTTP/1.1" 415 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 09:44:27] "POST /api/chat HTTP/1.1" 415 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 09:47:52] "POST /api/chat HTTP/1.1" 415 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 10:01:38] "POST /api/chat HTTP/1.1" 415 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 10:07:04] "POST /api/chat HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 10:07:20] "POST /api/chat HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 10:08:31] "POST /api/chat HTTP/1.1" 200 -
INFO

In [2]:
#yedek ngrok ile
from flask import Flask, request, Response
import threading
import requests
import time
from pyngrok import ngrok
from google.colab import userdata

# Flask app
app = Flask(__name__)

@app.route("/api/<path:path>", methods=["GET", "POST"])
def proxy(path):
    upstream_url = f"http://127.0.0.1:11434/api/{path}"
    if request.method == "POST":
        resp = requests.post(upstream_url, json=request.json)
    else:
        resp = requests.get(upstream_url)
    return Response(resp.content, status=resp.status_code, content_type=resp.headers['Content-Type'])

# Flask uygulamasını thread içinde çalıştır
def run_flask():
    app.run(port=5000)

flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

# ngrok tüneli aç
ngrok_key = userdata.get('ngrok_token')
ngrok.set_auth_token(ngrok_key)
public_url = ngrok.connect(5000).public_url
print(f"🔗 NGROK URL: {public_url}")

# Keep-alive döngüsü
while True:
    try:
        r = requests.get(public_url + "/api/tags")  # Ollama endpoint testi
        print("✅ Keep-alive başarılı:", r.status_code)
    except Exception as e:
        print("❌ Hata:", e)
    time.sleep(60)


ModuleNotFoundError: No module named 'pyngrok'

In [ ]:
!curl -X POST http://localhost:11434/api/chat -d '{ "model": "gemma3:27b",  "messages": [ { "role": "user", "content": "Merhaba! Nasılsın?" }  ]}'

{"model":"gemma3:27b","created_at":"2025-07-26T15:09:01.82619084Z","message":{"role":"assistant","content":"Merhaba"},"done":false}
{"model":"gemma3:27b","created_at":"2025-07-26T15:09:01.927247727Z","message":{"role":"assistant","content":"!"},"done":false}
{"model":"gemma3:27b","created_at":"2025-07-26T15:09:02.004022516Z","message":{"role":"assistant","content":" İ"},"done":false}
{"model":"gemma3:27b","created_at":"2025-07-26T15:09:02.081746934Z","message":{"role":"assistant","content":"y"},"done":false}
{"model":"gemma3:27b","created_at":"2025-07-26T15:09:02.158889698Z","message":{"role":"assistant","content":"iyim"},"done":false}
{"model":"gemma3:27b","created_at":"2025-07-26T15:09:02.235253488Z","message":{"role":"assistant","content":","},"done":false}
{"model":"gemma3:27b","created_at":"2025-07-26T15:09:02.311213569Z","message":{"role":"assistant","content":" teşekkür"},"done":false}
{"model":"gemma3:27b","created_at":"2025-07-26T15:09:02.386990427Z","message":{"role":"assista

In [ ]:
# Şimdi chat isteği gönderebiliriz
!curl -X POST http://127.0.0.1:11434/api/chat -d '{"model": "gemma3:27b",   "messages": [ { "role": "user", "content": "Merhaba! Nasılsın?" } ]}'

{"model":"gemma3:27b","created_at":"2025-06-01T21:27:43.172631868Z","message":{"role":"assistant","content":"Merhaba"},"done":false}
{"model":"gemma3:27b","created_at":"2025-06-01T21:27:43.273721013Z","message":{"role":"assistant","content":"!"},"done":false}
{"model":"gemma3:27b","created_at":"2025-06-01T21:27:43.349022989Z","message":{"role":"assistant","content":" Ben"},"done":false}
{"model":"gemma3:27b","created_at":"2025-06-01T21:27:43.424508783Z","message":{"role":"assistant","content":" iy"},"done":false}
{"model":"gemma3:27b","created_at":"2025-06-01T21:27:43.499983188Z","message":{"role":"assistant","content":"iyim"},"done":false}
{"model":"gemma3:27b","created_at":"2025-06-01T21:27:43.575728135Z","message":{"role":"assistant","content":","},"done":false}
{"model":"gemma3:27b","created_at":"2025-06-01T21:27:43.651248033Z","message":{"role":"assistant","content":" teşekkür"},"done":false}
{"model":"gemma3:27b","created_at":"2025-06-01T21:27:43.72687754Z","message":{"role":"ass

In [ ]:
!curl -X POST http://localhost:11434/api/generate -d '{  "model": "llama2",  "prompt":"Why is the sky blue?" }'

{"model":"llama2","created_at":"2025-06-01T21:26:54.439995664Z","response":"\n","done":false}
{"model":"llama2","created_at":"2025-06-01T21:26:54.46933021Z","response":"The","done":false}
{"model":"llama2","created_at":"2025-06-01T21:26:54.48715164Z","response":" sky","done":false}
{"model":"llama2","created_at":"2025-06-01T21:26:54.504606828Z","response":" appears","done":false}
{"model":"llama2","created_at":"2025-06-01T21:26:54.522495851Z","response":" blue","done":false}
{"model":"llama2","created_at":"2025-06-01T21:26:54.539972472Z","response":" because","done":false}
{"model":"llama2","created_at":"2025-06-01T21:26:54.557451367Z","response":" of","done":false}
{"model":"llama2","created_at":"2025-06-01T21:26:54.578793788Z","response":" a","done":false}
{"model":"llama2","created_at":"2025-06-01T21:26:54.596277266Z","response":" phenomen","done":false}
{"model":"llama2","created_at":"2025-06-01T21:26:54.61386867Z","response":"on","done":false}
{"model":"llama2","created_at":"2025-